### Julia + JuMP para optimización matemática



#### Instalar JuMP y GLPK

En primer lugar se agregará el paquete JuMP ejecutando el siguiente código en el Notebook, nótese que el simbolo # inicia un comentario en Julia.

In [1]:
# import Pkg # Importa el administrador de paquetes Pkg
# Pkg.add("JuMP") # Instala el paquete JuMP

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


Es necesario instalar un Solver (paquete con métodos de optimización implementados, por ejemplo Simplex). Se instalará el Solver open source GLPK (pueden encontrar info en https://www.gnu.org/software/glpk/) escribiendo el siguiente comando.

In [2]:
# Pkg.add("GLPK") # Instala el paquete GLPK

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


#### Ejemplo nro. 1 
Se resuelve el siguiente problema de programación lineal utilizando JuMP para verificar su funcionamiento.

$$ maximizar \quad 3x_{1} + 2x_{2} $$
$$ s.a. \quad 2x_{1} + x_{2} \leq 100 $$ 
$$ x_{1} + x_{2} \leq 80  $$
$$ x_{1} \geq 0, x_{2} \geq 0 $$ 

In [ ]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
#modeloUno = Model(with_optimizer(GLPK.Optimizer)) # Instancia un modelo. Las variables y restricciones son asociadas con un modelo.
# El modelo creado se encuentra asociado a un solver, en este caso GLPK. 

modeloUno = Model(GLPK.Optimizer)

# VARIABLES DE DECISIÓN
# Una variable es modelada utilizando @variable(nombre del modelo, nombre de la variable y límites, tipo de la variable). 
# El límite puede ser superior, inferior o ambos. Si no se define un tipo, entonces la variable es tratada como real. 
# Para variables enteras se utiliza Int y para variables binarias Bin.  
@variable(modeloUno, x >= 0) # Modela la variable continua x.
@variable(modeloUno, y >= 0) # Modela la variable continua y.

# Agunos ejemplos:
# @variable(nombre modelo, x, Bin) # Modela la variable binaria x, nótese que no se utiliza límites.
# @variable(myModel, x <= 10) # Modela una variable continua con límite suprior. 
# @variable(myModel, 0 <= x <= 10, Int) # Modela una variable entera con límite superior e inferior.


# FUNCIÓN OBJETIVO
@objective(modeloUno, Max, 3x + 2y) # Establece la maximización de la función objetivo. Para minimizar, utilizar Min.

# RESTRICCIONES
@constraint(modeloUno, 2x + y <= 100) # Agrega una restriccción al modelo.
@constraint(modeloUno, x + y <= 80) # Agrega una restriccción al modelo.

optimize!(modeloUno) # Resuelve el  modelo
objective_value(modeloUno) # Entrega el valor objetivo óptimo

In [ ]:
termination_status(modeloUno) # Entrega valor de estado de término del modelo. 1 = óptimo; 2 = infactible.

In [ ]:
value(x) # Entrega el valor de la variable de decisión x.

In [ ]:
value(y) # Entrega el valor de la variable de decisión y.

#### Ejemplo nro. 1 de otra forma
Se resuelve el mismo problema utilizando vectores para los parámetros (costos, lado derecho, coeficientes tecnológicos) y para las variables de decisión.  
$$ maximizar \quad 3x_{1} + 2x_{2} $$ 
$$ s.a. \quad 2x_{1} + x_{2} \leq 100 $$
$$ x_{1} + x_{2} \leq 80  $$
$$ x_{1} \geq 0, x_{2} \geq 0 $$

In [ ]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
modeloDos = Model(GLPK.Optimizer) # Instancia un modelo.

# PARÁMETROS DE PROBLEMA
c = [3; 2] # Construye un arreglo de dimensiones 2 (fila) x 1 (columna).
A = [2 1; # Construye un arreglo de dimensiones 2 (fila) x 2 (columna).
     1 1]
b = [100; 80] # Construye un vector de dimensiones 2 (fila) x 1 (columna).

m,n = size(A) # Entrega las dimensiones del arreglo
# VARIABLES DE DECISIÓN
@variable(modeloDos, x[1:n] >= 0) # Modela un arreglo de variables x de dimensiones 1 (fila) x 2 (columna).

# FUNCIÓN OBJETIVO
@objective(modeloDos, Max, sum(c[i]*x[i] for i in 1:n)) # Establece la maximización de la función objetivo.
# Utiliza la función sum() para definir la función objetivo mediante una sumatoria.

# RESTRICCIONES
for i in 1:m
    @constraint(modeloDos, sum(A[i,j]*x[j] for j in 1:n) <= b[i]) # Agrega dos restriccción al modelo.
end
    
optimize!(modeloDos) # Resuelve el  modelo
objective_value(modeloDos) # Entrega el valor objetivo óptimo

In [ ]:
for i in 1:n
    println("Variable X",i,": ",value(x[i]))
end

#### Ejemplo nro. 2
Un fabricante mantiene tres centros de distribución: Cerrillos, Peñalolén y Quilicura. Estos centros tienen disponibilidades de: 2000, 4000 y 4000 unidades respectivamente. Sus detallistas requieren las siguientes cantidades: Maipú 2500, Estación Central 1000, Santiago 2000, Providencia 3000 y La Reina 1500. El costo de transporte por unidad en pesos entre cada centro de distribución y las comunas de los detallistas se dan en la tabla:

 _   |Maipú |Est. Central |Santiago |Providencia |La Reina
-----|-----|-----|-----|-----|-----
Cerrillos|55|30|40|50|40
Peñalolén|35|30|100|45|60
Quilicura|40|60|95|35|30

Formule un modelo para determinar el número de unidades que debe enviar el fabricante desde cada centro de distribución a cada detallista, de manera que los costos totales de transporte sean mínimos.

=============================================================================================================================

Modelo:


Variables de decisión:

$x_{ij}:$ cantidad de unidades enviadas desde el centro i-ésimo al detallista j-ésimo; con i = 1,2,3 y j = 1,...,5

Función objetivo:

$minimizar\, \sum_{i=1}^{3}\sum_{j=1}^{5}c_{ij}x_{ij}$

Restricciones:

$o_{i}:$ oferta del punto de distribución i-ésimo

$d_{j}:$ demanda del punto detallista j-ésimo

$\sum_{j=1}^{5}x_{ij} \leq o_{i}$; con i = 1,2,3 

$\sum_{i=1}^{3}x_{ij} \geq d_{j}$; con j = 1,...,5

Restricciones de no negatividad:

$x_{ij} \geq 0$; con i = 1,2,3 y j = 1,...,5



In [ ]:
using JuMP
using GLPK

# CONSTRUCCIÓN DEL MODELO
modeloTres = Model(GLPK.Optimizer)

# PARÁMETROS DE PROBLEMA
c = [55 30 40 50 40; 
    35 30 100 45 60; 
    40 60 95 35 30]


A = [1 1 1 1 1 0 0 0 0 0 0 0 0 0 0;
     0 0 0 0 0 1 1 1 1 1 0 0 0 0 0;
     0 0 0 0 0 0 0 0 0 0 1 1 1 1 1;
     1 0 0 0 0 1 0 0 0 0 1 0 0 0 0;
     0 1 0 0 0 0 1 0 0 0 0 1 0 0 0;
     0 0 1 0 0 0 0 1 0 0 0 0 1 0 0;
     0 0 0 1 0 0 0 0 1 0 0 0 0 1 0;
     0 0 0 0 1 0 0 0 0 1 0 0 0 0 1]

b = [2000; 4000; 4000; 2500; 1000; 2000; 3000; 1500]

m,n = size(A)

# VARIABLES DE DECISIÓN
@variable(modeloTres, x[1:3, 1:5] >= 0)

# FUNCIÓN OBJETIVO
@objective(modeloTres, Min, sum(c[i,j]*x[i,j] for i in 1:3, j in 1:5))

# RESTRICCIONES
for i in 1:m
    @constraint(modeloTres, sum(A[i, (j * 5 + k)]*x[(j + 1),k] for j in 0:2, k in 1:5) == b[i])
end

 
optimize!(modeloTres)
objective_value(modeloTres)

In [ ]:
for i in 1:3
    for j in 1:5
        println("Variable X",i,j," = ",value(x[i,j]))
    end
end

#### Ejemplo nro. 3
La industria papelera DIMAR que fabrica papel y lo distribuye en rollos debe determinar la mejor forma de realizar el proceso de corte. Los rollos de papel que se producen tienen un ancho de 100 cm; sin embargo, algunas librerías demandan rollos de 30 cm, 45 cm y 50 cm de ancho. Por lo tanto, al cortar los rollos de 100 cm se incurre en una pérdida de material que depende de la forma en que se corten los rollos originales. 

Se desea determinar la forma de efectuar el corte de manera que se satisfaga la demanda y se minimice la pérdida total de material. Se tiene un pedido de 800 rollos de 30 cm de ancho, 500 rollos de 45 cm y 1.000 rollos de 50 cm. Plantee un modelo de PL que minimice las perdidas.

=============================================================================================================================

Modelo:

-|Esquema de corte | pérdida 
-----|-----|-----
1|30-30-30|10
2|30-45|25
3|30-50|20
4|45-45|10
5|45-50|5
6|50-50|0




Variables de decisión:

$x_{i}:$ número de rollos cortados según el esquema i; con $i = 1,2,3,4,5,6$

Función objetivo:

$minimizar\, 10x_{1}+25x_{2}+20x_{3}+10x_{4}+5x_{5}$

Restricciones:

$3x_{1}+x_{2}+x_{3} \geq 800$

$x_{2}+2x_{4}+x_{5} \geq 500$

$x_{3}+x_{4}+2x_{6} \geq 1000$

Restricciones de no negatividad:

$x_{1},x_{2},x_{3},x_{4},x_{5},x_{6} \geq 0$

In [ ]:
using JuMP
using GLPK

# CONSTRUCCIÓN DEL MODELO
modeloCuatro = Model(GLPK.Optimizer)

# PARÁMETROS DE PROBLEMA
c = [10 25 20 10 5 0]


A = [3 1 1 0 0 0;
     0 1 0 2 1 0;
     0 0 1 1 0 2]

b = [800; 500; 1000]


# VARIABLES DE DECISIÓN
@variable(modeloCuatro, x[1:6] >= 0, Int)

# FUNCIÓN OBJETIVO
@objective(modeloCuatro, Min, sum(c[i]*x[i] for i in 1:6))

# RESTRICCIONES
for i in 1:3
    @constraint(modeloCuatro, sum(A[i,j]*x[j] for j in 1:6) >= b[i])
end

 
optimize!(modeloCuatro)
objective_value(modeloCuatro)

In [ ]:
for j in 1:6
    println("Variable X",j," = ",value(x[j]))
end

#### Ejemplo nro. 4
Una empresa siderúrgica ha recibido un pedido de 500 toneladas de acero destinado a la
construcción naval. El acero, debe cumplir con los siguientes requisitos porcentuales en su
composición:

<center>Tabla 2: Requisitos porcentuales.</center>

| Elemento químico 	| Mínimo Porcentual (%)  | Máximo Porcentual (%) |
|:-----------------:|:-----------------:|:-----------------:|
| Carbón (C)     	| 2            	    | 3         	    |
| Cobre (Cu)      	| 0.4 	            | 0.6         	    |
| Manganeso (Mn)    | 1.2     	        | 1.65              |

La empresa cuenta con siete materias primas diferentes en stock que pueden ser utilizadas
para la producción del acero. En tabla nro. 3 se enumeran los porcentajes, las cantidades
disponibles y los precios de todas las materias primas:

<center>Tabla 3: Materias prima y costos.</center>

| Materia prima     | C %  | Cu % | Mn % | Disponibilidad en t | Costo en €/t
|:-----------------:|:----:|:----:|:----:|:-------------------:|:-----------:
| Hierro 1          | 2.5  | 0    | 1.3  | 400                 | 200
| Hierro 2          | 3    | 0    | 0.8  | 300                 | 250
| Hierro 3          | 0    | 0.3  | 0    | 600                 | 150
| Cobre 1           | 0    | 90   | 0    | 500                 | 220
| Cobre 2           | 0    | 96   | 4    | 200                 | 240
| Aluminio 1        | 0    | 0.4  | 1.2  | 300                 | 200
| Aluminio 2        | 0    | 0.6  | 0    | 250                 | 165

El objetivo es determinar la composición de la aleación de acero que minimice el costo de
producción.

=============================================================================================================================

Modelo:

Variables de decisión:

$x_{i}:$ cantidad de materia prima tipo $\textit{i}$ utilizada en la aleación; con i = 1,...,7

Función objetivo: (minimizar costos)

$minimizar\,200x_{1}+250x_{2}+150x_{3}+220x_{4}+240x_{5}+200x_{6}+165x_{7}$

Restricciones:

$0.025x_{1}+0.03x_{2} \geq 0.02 \cdot 500$

$0.003x_{3}+0.9x_{4}+0.96x_{5}+0.004x_{6}+0.006x_{7} \geq 0.004 \cdot 500$

$0.013x_{1}+0.008x_{2}+0.04x_{5}+0.012x_{6} \geq 0.012 \cdot 500$

$0.025x_{1}+0.03x_{2} \leq 0.03 \cdot 500$

$0.003x_{3}+0.9x_{4}+0.96x_{5}+0.004x_{6}+0.006x_{7} \leq 0.006 \cdot 500$

$0.013x_{1}+0.008x_{2}+0.04x_{5}+0.012x_{6} \leq 0.0165 \cdot 500$

$x_{1}+x_{2}+x_{3}+x_{4}+x_{5}+x_{6}+x_{7}=500$


Restricciones de disponibilidad:

$x_{1} \leq 400 ,x_{2} \leq 300,x_{3} \leq 600,x_{4} \leq 500,x_{5} \leq 200,x_{6} \leq 300,x_{7} \leq 250$

Restricciones de no negatividad:

$x_{1},x_{2},x_{3},x_{4},x_{5},x_{6},x_{7} \geq 0$





In [ ]:
using JuMP # Importa el módulo JuMP.
using GLPK # Importa el módulo GLPK para utilizar el solver.

# CONSTRUCCIÓN DEL MODELO
modeloCinco = Model(GLPK.Optimizer) # Instancia un modelo.

# PARÁMETROS DE PROBLEMA
porcentual = [0.02 0.03; 0.004 0.006; 0.012 0.0165] 

materiaPrima = [0.025 0 0.013 400 200;
0.03 0 0.008 300 250;
0 0.003 0 600 150;
0 0.9 0 500 220;
0 0.96 0.04 200 240;
0 0.004 0.012 300 200;
0 0.006 0 250 165]


m,n = size(materiaPrima) # Entrega las dimensiones del arreglo
# VARIABLES DE DECISIÓN
@variable(modeloCinco, x[1:m] >= 0) # Modela un arreglo de variables x de dimensiones 1 x m.

# FUNCIÓN OBJETIVO
@objective(modeloCinco, Min, sum(materiaPrima[i, 5] * x[i] for i in 1:m)) # Establece la minimización de la función objetivo.
# Utiliza la función sum() para definir la función objetivo mediante una sumatoria.

# RESTRICCIONES
for j in 1:3
    @constraint(modeloCinco, sum(materiaPrima[i, j] * x[i] for i in 1:m) >= 500 * porcentual[j, 1]) # Agrega tres restriccción al modelo    
end

for j in 1:3
    @constraint(modeloCinco, sum(materiaPrima[i, j] * x[i] for i in 1:m) <= 500 * porcentual[j, 2]) # Agrega tres restriccción al modelo    
end

for i in 1:m
    @constraint(modeloCinco, x[i] <= materiaPrima[i, 4])
end


@constraint(modeloCinco, sum(x[i] for i in 1:m) >= 500)  


optimize!(modeloCinco) # Resuelve el  modelo
println("Valor objetivo = ", objective_value(modeloCinco)) # Entrega el valor objetivo óptimo

In [ ]:
for j in 1:m
    println("Variable X",j," = ",value(x[j]))
end

#### Ejemplo nro. 5 
Problema nro. 3 de la guía nro. 1.

=============================================================================================================================

Modelo:

Variables de decisión:

$x_{i}:$ 1 se extrae el bloque. 0 no se extrae el bloque. Con i = 1,...,18

Función objetivo: (minimizar costos)

$$ minimizar \quad \sum_{t=1}^{18}(v_{t}- c_{t}) * x_{tp} $$ 

Restricciones:

$$ \quad x_{a}-x_{b} \leq 0 \quad \forall (a, b) \in A  $$ 

In [ ]:
########### lectura de parámetros desde un archivo ###########

f =open("data_problema_5.csv") 
# Costo_extraccion 
costoExtraccion = parse.(Int, split(chop(readline(f)), ' '))
# Valor de cada bloque
valorBloque = parse.(Int, split(chop(readline(f)), ' '))
g = readline(f)
# Grafo como matriz de adyacencia
G = [parse(Int, ss) for ss in split(g)]
while ! eof(f) 
	s = readline(f)	
    r = [parse(Int, ss) for ss in split(s)]
    G = vcat(G, r) 
end
G = transpose(reshape(G, (18, 18)))

########### Fin lectura archivo ###########
# CONSTRUCCIÓN DEL MODELO

modeloSeis = Model(GLPK.Optimizer)

# VARIABLES DE DECISIÓN
@variable(modeloSeis, x[1:18], Bin) # Por cada bloque, 1 si es extraído y 0 si no

# FUNCIÓN OBJETIVO

# Para cada bloque a extraer, se suma la diferencia entre el valor de cada bloque y el costo de extracción
@objective(modeloSeis, Max, sum((valorBloque[i] - costoExtraccion[i]) * x[i] * 10000 for i in 1:18)) 

# RESTRICCIONES
# Se Recorre el grafo de dependencias
for i in 1:18
    for j in 1:18
        if G[i, j] == 1
        # Se aplican las dependencias de extracción de bloques        
            @constraint(modeloSeis, (x[i] - x[j]) <= 0)
        end
    end
end

# SOLUCIÖN DEL MODELO
optimize!(modeloSeis)
println("Valor objetivo = ", objective_value(modeloSeis))
for i in 1:18
    println("Variable X",i," = ",value(x[i]))
end